# 이웃집 토토치 파이토치 : Day 3
---

<div class="alert alert-info">
    <p>📢 해당 게시물은 <a href="https://tutorials.pytorch.kr/beginner/basics/intro.html">파이토치 기본 익히기</a>와 torchvision에서 제공하는 <a href="https://github.com/pytorch/vision/blob/main/torchvision/models/vgg.py">vgg 모델</a>을 재구성하여 작성되었습니다.</p>
</div>

## VGG16 작성 및 학습

![](https://miro.medium.com/max/700/1*gU5m4XO2awEM6Zp4DkirFA.png)

오늘은 VGG16 모델을 CIFAR-10 데이터를 학습 시켜볼 수 있는 이미지 분류기로 정의하고 이를 학습시켜 보고자 합니다.

작성해야하는 분류기와 학습 환경에 대한 조건은 다음과 같습니다.

- transforms : 정규화만을 진행한다.
- 모델 구조
    - feature [64, 64, M, 128, 128, M, 256, 256, 256, M, 512, 512, 512, M, 512, 512, 512, M] + classifier [AA, 4096, 4096, 1000]
    - feature
        - n(숫자) : filter의 수가 n개인 2d Conv Layer, `kernel_size=3`, `padding=1`
        - M : Max Pooling Layer, `kernel_size=2`, `padding=2`
        - Conv Layer 이후의 activation func는 ReLU를 사용한다.
    - classifier
        - AA : Adaptive Average Pooling(7x7)
        - n(숫자) : Linear(n unit)
- loss 함수 : CrossEntropyLoss
- optimizer = SGD, lr = 0.005, momentum=0.9
    - StepLR를 사용하여 5 step 마다 lr를 0.9배씩 줄이며 학습시킨다.
- 학습 과정에서 train set과 test set에 대한 loss와 accuracy를 출력하고, 학습 종료 후 matrix의 변화를 plot으로 출력한다.
- 학습이 완료된 모델을 `my_vgg16`이라는 이름으로 [저장](https://tutorials.pytorch.kr/beginner/basics/saveloadrun_tutorial.html)한다.

In [ ]:
!pip install torchinfo

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchinfo import summary

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# parameters
lr = 0.005
epochs = 15
batch_size = 128

In [ ]:
transform = transforms.Compose([ # ransforms.Compose() : 여러 변환을 함께 구성
    transforms.ToTensor(), # .ToTensor() : PIL, Image or numpy.ndarray을 tensor로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # .Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) : 평균(0.5, 0.5, 0.5) 및 표준 편차(0.5, 0.5, 0.5)를 사용하여 부동 텐서 이미지를 정규화
])

train_loader = DataLoader(
    datasets.CIFAR10(root='./data', train=True, transform=transform, download=True),
    batch_size=batch_size, shuffle=True)

val_loader = torch.utils.data.DataLoader( # torch.utils.data.DataLoader : 데이터 로더. 데이터 세트와 샘플러를 결합하고 주어진 데이터 세트에 대해 반복 가능을 제공
    datasets.CIFAR10(root='./data', train=False, transform=transform),
    batch_size=batch_size, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            # 이곳을 구현해 주세요
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=64, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, padding=2),
        )

        self.classifier = nn.Sequential(
            # 이곳을 구현해 주세요
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Linear(512*2*2, 4096),
            nn.Linear(4096, 4096),
            nn.Linear(4096, 1000),
        )

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            x = self.features(x)
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.classifier(x)
            return x

In [ ]:
criterion = nn.CrossEntropyLoss() # loss func를 정의하여 주세요
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)# optimizer를 정의하여 주세요
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)# learning scheduler를 정의하여 주세요

NameError: ignored

In [ ]:
for epoch in range(epochs): 
    # 이곳을 구현해 주세요

SyntaxError: ignored

In [ ]:
# 학습경과 출력
# 이곳을 구현해 주세요

In [ ]:
# 모델 저장
# 이곳을 구현해 주세요